In [25]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import random
import copy
%matplotlib inline

In [2]:
user_data = pd.read_csv('17_Number_data.csv', index_col=0)
user_data

,MP61026,MP61273,MP61034,MP61040,MP61228,MP61166,MP61171,MP61222,MP61275,MP61027,MP61255,MP61021,MP61043,MP61151,MP61172,MP61223
IDSTUD,,,,,,,,,,,,,,,,
50030203,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0.0,1.0,1,1.0,1.0,1
50030211,1,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,0.0,1.0,0,1.0,0.0,1
50040408,1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1,1.0,0.0,1,1.0,1.0,1
50040416,1,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0,0.0,0.0,1,1.0,1.0,1
50040424,1,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,1.0,0.0,1,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51680206,1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1,0.0,0.0,1,NaN,0.0,1
51680214,1,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1,1.0,1.0,1,1.0,1.0,1
51690506,1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,1.0,0.0,1,1.0,1.0,1


In [3]:
user_dict = user_data.to_dict('index')
users = user_data.axes[0].tolist()
items = user_data.columns.tolist()
print(user_dict[50030203])

{' MP61026': 1, ' MP61273': 1.0, ' MP61034': 1.0, ' MP61040': 1.0, ' MP61228': 1.0, ' MP61166': 1.0, ' MP61171': 1.0, ' MP61222': 1.0, ' MP61275': 1.0, ' MP61027': 0, ' MP61255': 0.0, ' MP61021': 1.0, ' MP61043': 1, ' MP61151': 1.0, ' MP61172': 1.0, ' MP61223': 1}


In [8]:
item_data = user_data.transpose()
item_dict = item_data.to_dict('index')
print(item_dict[' MP61026'])

{50030203: 1.0, 50030211: 1.0, 50040408: 1.0, 50040416: 1.0, 50040424: 1.0, 50060207: 1.0, 50060215: 1.0, 50080207: 1.0, 50080215: 1.0, 50080223: 1.0, 50090604: 1.0, 50090612: 1.0, 50090620: 1.0, 50090628: 0.0, 50100106: 1.0, 50100114: 1.0, 50100122: 1.0, 50110508: 1.0, 50110516: 1.0, 50150302: 1.0, 50220306: 0.0, 50220314: 1.0, 50220322: 0.0, 50230102: 1.0, 50250404: 1.0, 50250412: 1.0, 50250420: 1.0, 50260216: 1.0, 50260224: 1.0, 50270902: 1.0, 50270910: 1.0, 50270918: 1.0, 50270926: 1.0, 50280202: 1.0, 50280210: 1.0, 50280218: 1.0, 50280226: 1.0, 50280234: 1.0, 50330502: 1.0, 50330510: 1.0, 50330518: 1.0, 50350302: 1.0, 50350310: 0.0, 50350318: 1.0, 50400508: 1.0, 50400516: 1.0, 50400524: 1.0, 50420405: 1.0, 50420413: 1.0, 50420421: 1.0, 50420429: 1.0, 50430303: 1.0, 50430311: 1.0, 50430319: 1.0, 50450103: 0.0, 50450111: 1.0, 50450119: 1.0, 50450127: 1.0, 50460110: 1.0, 50460118: 1.0, 50460126: 1.0, 50520108: 1.0, 50520116: 1.0, 50520124: 1.0, 50570104: 1.0, 50570112: 1.0, 50570120:

In [30]:
def cosine_sim_user(user1, user2, dictionary):
    sumsq1 = 0
    sumsq2 = 0
    innerprod = 0
    for item in items:
        rating1 = dictionary[user1][item]
        rating2 = dictionary[user2][item]
        if not np.isnan(rating1) and not np.isnan(rating2):
            sumsq1 += pow(rating1, 2)
            sumsq2 += pow(rating2, 2)
            innerprod += rating1 * rating2
    if sumsq1 * sumsq2 == 0:
        print(user1, sumsq1, user2, sumsq2)
    return innerprod / (sqrt(sumsq1) * sqrt(sumsq2))

def cosine_sim_item(item1, item2, dictionary):
    sumsq1 = 0
    sumsq2 = 0
    innerprod = 0
    for user in users:
        rating1 = dictionary[item1][user]
        rating2 = dictionary[item2][user]
        if not np.isnan(rating1) and not np.isnan(rating2):
            sumsq1 += pow(rating1, 2)
            sumsq2 += pow(rating2, 2)
            innerprod += rating1 * rating2
    return innerprod / (sqrt(sumsq1) * sqrt(sumsq2))

def cosine_sim_avg_user(avg, user2, dictionary):
    sumsq1 = 0
    sumsq2 = 0
    innerprod = 0
    for item in items:
        rating1 = avg[item]
        rating2 = dictionary[user2][item]
        if not np.isnan(rating1) and not np.isnan(rating2):
            sumsq1 += pow(rating1, 2)
            sumsq2 += pow(rating2, 2)
            innerprod += rating1 * rating2
    return innerprod / (sqrt(sumsq1) * sqrt(sumsq2))

def cosine_sim_avg_item(avg, item2, dictionary):
    sumsq1 = 0
    sumsq2 = 0
    innerprod = 0
    for user in users:
        rating1 = avg[user]
        rating2 = dictionary[item2][user]
        if not np.isnan(rating1) and not np.isnan(rating2):
            sumsq1 += pow(rating1, 2)
            sumsq2 += pow(rating2, 2)
            innerprod += rating1 * rating2
    return innerprod / (sqrt(sumsq1) * sqrt(sumsq2))

def similarity(obj, obj1, obj2, dictionary):
    if obj == 'user':
        return cosine_sim_user(obj1, obj2, dictionary)
    elif obj == 'item':
        return cosine_sim_item(obj1, obj2, dictionary)
    else:
        raise ValueError

def similarity_avg(obj, avg, obj2, dictionary):
    if obj == 'user':
        return cosine_sim_avg_user(avg, obj2, dictionary)
    elif obj == 'item':
        return cosine_sim_avg_item(avg, obj2, dictionary)
    else:
        raise ValueError

In [5]:
def clustering(obj, users, items, dictionary, k):
    simname = 'cosine'
    center_users = users[:k]
    while True:
        clusters = {}
        for i in range(k):
            clusters[center_users[i]] = []

        for user in users:
            max_sim = -1
            max_sim_center_user = []
            for center_user in center_users:
                sim = similarity(obj, user, center_user, dictionary)
                if sim > max_sim:
                    max_sim_center_user = [center_user]
                    max_sim = sim
                elif sim == max_sim:
                    max_sim_center_user.append(center_user)
            cnt = len(max_sim_center_user)
            chosen = max_sim_center_user[random.randint(0, cnt - 1)]
            clusters[chosen].append(user)

        new_center_users = []
        for cluster_center in clusters:
            cluster = clusters[cluster_center]
            average = {}
            for item in items:
                sum = 0
                count = 0
                for user in cluster:
                    rating = dictionary[user][item]
                    if not np.isnan(rating):
                        sum += rating
                        count += 1
                if count == 0:
                    average[item] = float('NaN')
                else: 
                    average[item] = sum / count

            max_sim = -1
            max_sim_user = -1
            for user in cluster:
                sim = similarity_avg(obj, average, user, dictionary)
                if sim > max_sim:
                    max_sim_user = user
                    max_sim = sim
            new_center_users.append(max_sim_user)

        if set(new_center_users) == set(center_users):
            break        

        center_users = new_center_users
    return clusters

In [182]:
## User Clustering
user_cluster = clustering('user', users, items, 5)
print(user_cluster)

{51470108: [50030203, 50060207, 50060215, 50080223, 50090604, 50090620, 50100114, 50100122, 50110508, 50150302, 50220322, 50230102, 50250404, 50250420, 50260224, 50270918, 50280202, 50280218, 50280226, 50280234, 50330518, 50350318, 50400508, 50400516, 50400524, 50420405, 50420413, 50420421, 50420429, 50430311, 50430319, 50450119, 50450127, 50460126, 50520108, 50520116, 50520124, 50570104, 50610403, 50610411, 50620115, 50630402, 50630410, 50630418, 50720502, 50720510, 50740522, 50790110, 50790118, 50820816, 50840511, 50840519, 50860406, 50860414, 50860422, 50890119, 50920507, 50920523, 50960209, 50960217, 50970502, 50970518, 50970526, 50980303, 50980311, 51030112, 51060311, 51060319, 51060327, 51150401, 51150409, 51180401, 51180409, 51180417, 51190316, 51210105, 51220310, 51220318, 51300107, 51330805, 51330813, 51340710, 51340718, 51350101, 51410513, 51410521, 51420405, 51420413, 51430201, 51430209, 51430217, 51470108, 51470124, 51480706, 51480722, 51500507, 51500515, 51500523, 51540308

In [183]:
## Fill Empty
for cluster_center in clusters:
    cluster = clusters[cluster_center]
    for item in items:
        empty_list = []
        sum = 0
        count = 0
        for user in clusters:
            rating = user_dict[user][item]
            if np.isnan(rating):
                empty_list.append(user)
            else:
                sum += rating
                count += 1
        average = sum / count
        for empty in empty_list:
            item_dict[item][empty] = average

In [184]:
## Item Clustering
item_cluster = clustering('item', items, users, 2)
print(item_cluster)

{' MP61026': [' MP61026', ' MP61034', ' MP61040', ' MP61228', ' MP61166', ' MP61171', ' MP61222', ' MP61275', ' MP61027', ' MP61255', ' MP61043', ' MP61151', ' MP61172', ' MP61223'], ' MP61273': [' MP61273', ' MP61021']}


In [175]:
## Collaborative Filtering & Evaluation
random.shuffle(users)
print(users)

[50030203, 50030211, 50040408, 50040416, 50040424, 50060207, 50060215, 50080207, 50080215, 50080223, 50090604, 50090612, 50090620, 50090628, 50100106, 50100114, 50100122, 50110508, 50110516, 50150302, 50220306, 50220314, 50220322, 50230102, 50250404, 50250412, 50250420, 50260216, 50260224, 50270902, 50270910, 50270918, 50270926, 50280202, 50280210, 50280218, 50280226, 50280234, 50330502, 50330510, 50330518, 50350302, 50350310, 50350318, 50400508, 50400516, 50400524, 50420405, 50420413, 50420421, 50420429, 50430303, 50430311, 50430319, 50450103, 50450111, 50450119, 50450127, 50460110, 50460118, 50460126, 50520108, 50520116, 50520124, 50570104, 50570112, 50570120, 50580203, 50580211, 50610403, 50610411, 50610419, 50620107, 50620115, 50630402, 50630410, 50630418, 50720502, 50720510, 50720518, 50730204, 50730212, 50730220, 50740506, 50740514, 50740522, 50790110, 50790118, 50820808, 50820816, 50820824, 50840503, 50840511, 50840519, 50860406, 50860414, 50860422, 50890103, 50890111, 50890119,

In [59]:
## Cross Validation
org_user_data = pd.read_csv('17_Number_data.csv', index_col=0)
org_user_dict = org_user_data.to_dict('index')
org_item_data = user_data.transpose()
org_item_dict = org_item_data.to_dict('index')

user_fold = 10
item_fold = 2
user_p = [0, 20, 40, 60, 80, 100, 120, 140, 160, 179, 198]
item_p = [0, 8, 16]

total_sum = 0
total_accuracy = 0
total_precision = 0
total_recall = 0
total_count = user_fold * item_fold
for i in range(user_fold):
    for j in range(item_fold):
        print(f'round{i}{j}')
        train_user_dict = copy.deepcopy(org_user_dict)
        train_item_dict = copy.deepcopy(org_item_dict)
        test_users = users[user_p[i]:user_p[i+1]]
        test_items = items[item_p[j]:item_p[j+1]]
        train_users = [user for user in users if user not in test_users]
        train_items = [item for item in items if item not in test_items]
        for user in test_users:
            for item in test_items:
                train_user_dict[user][item] = float('NaN')
                train_item_dict[item][user] = float('NaN')

        ## User Clustering
        user_cluster = clustering('user', users, items, train_user_dict, 5)
        ## Fill Empty
        for cluster_center in user_cluster:
            cluster = user_cluster[cluster_center]
            for item in train_items:
                empty_list = []
                sum = 0
                count = 0
                for user in cluster:
                    rating = train_user_dict[user][item]
                    if user not in test_users:
                        if np.isnan(rating):
                            empty_list.append(user)
                        else:
                            sum += rating
                            count += 1
                if count == 0: continue
                average = sum / count
                for empty in empty_list:
                    train_item_dict[item][empty] = average
        ## Item Clustering
        item_cluster = clustering('item', items, users, train_item_dict, 4)
        ## Collaborative Filtering
        accuracy_count = 0
        precision_count = 0
        recall_count = 0
        pos_count = 0
        neg_count = 0
        rating_sum = 0
        rating_count = 0
        for test_item in test_items:
            max_sim = -1
            sim_items = []
            for cluster_center in item_cluster:
                sim = similarity('item', test_item, cluster_center, train_item_dict)
                if sim > max_sim:
                    sim_items = item_cluster[cluster_center]
                for test_user in test_users:
                    sim_rating_sum = 0
                    sim_sum = 0
                    for compare_item in sim_items:
                        rating = train_item_dict[compare_item][test_user]
                        if compare_item in train_items and not np.isnan(rating):
                            sim = similarity('item', test_item, compare_item, train_item_dict)
                            sim_sum += sim
                            sim_rating_sum += sim * rating
                    if sim_sum == 0: continue
                    prediction = sim_rating_sum / sim_sum
                    real_rating = org_user_dict[test_user][test_item]
                    if not np.isnan(real_rating):
                        rating_sum += abs(prediction - real_rating)
                        rating_count += 1
                        if prediction >= 0.5:
                            pos_count += 1
                        if real_rating == 1:
                            neg_count += 1
                        if prediction < 0.5 and real_rating == 0:
                            accuracy_count += 1
                        if prediction >= 0.5 and real_rating == 1:
                            accuracy_count += 1
                            precision_count += 1
                            recall_count += 1
                        
        if rating_count == 0: continue
        total_accuracy += accuracy_count / rating_count
        total_precision += precision_count / pos_count
        total_recall += recall_count / neg_count
        tmp_result = rating_sum / rating_count
        total_sum += tmp_result

a_result = total_accuracy / total_count
p_result = total_precision / total_count
r_result = total_recall / total_count
MAE = total_sum / total_count
print('-------------------------------------------')
print(f'MAE: {MAE}')
print(f'precision: {p_result}')  
print(f'recall: {r_result}')  
print(f'accuracy: {a_result}')  

round00
round01
round10
round11
round20
round21
round30
round31
round40
round41
round50
round51
round60
round61
round70
round71
round80
round81
round90
round91
-------------------------------------------
MAE: 0.3138715605220808
precision: 0.8021462463026057
recall: 0.8405504811134501
accuracy: 0.7255811629623188
